In [ ]:
data <- read.csv("C:/Users/Sean/Desktop/courseworkdata.csv")  # Read CSV file into R

In [ ]:
install.packages('MASS', repos="http://cran.us.r-project.org")
library(MASS)  # Load package required for mvrnorm function
install.packages('reshape2', repos="http://cran.us.r-project.org")
library(reshape2)  # Load package required for melt function

In [ ]:
e <- data$touchX[1:300] - data$targetX[1:300]  # Define offset for X coordinates 
f <- data$touchY[1:300] - data$targetY[1:300]  # Define offset for Y coordinates 
z <- c(e, f)  # Combine offset values for Subject15-iPhone4 into single vector

In [ ]:
CalculateCovariance <- function(X1, X2, l=1) {
  sigma <- matrix(rep(0, length(X1) * length(X2)), nrow=length(X1))
  alpha <- 0.3
  for (i in 1:300) {
    for (j in 1:300) {
      sigma[i, j] <- exp(-10 * (abs(X1[i] - X2[j]) / l)^2)  # Squared exponential kernel function
    }
  }
  for (i in 301:600) {
    for (j in 301:600) {
      sigma[i, j] <- exp(-10 * (abs(X1[i] - X2[j]) / l)^2)
    }
  }
  for (i in 1:300) {
    for (j in 301:600) {
      sigma[i, j] <- exp(-10 * (abs(X1[i]-  X2[j]) / l)^2) * alpha
    }
  }
  for (i in 301:600) {
    for (j in 1:300) {
      sigma[i, j] <- exp(-10 * (abs(X1[i] - X2[j]) / l)^2) * alpha
    }
  }
  return(sigma)
}
# Function CalculateCovariance returns the covariance matrix 

In [ ]:
sigma.one <- CalculateCovariance(z, z)  # Define covariance matrix for training data

In [ ]:
gaussian.noise <- 0.1  # Define additive Gaussian noise

In [ ]:
gp.mean.one <- sigma.one %*% solve(sigma.one + gaussian.noise^2 * diag(1, ncol(sigma.one))) %*% z
gp.covariance.one <- sigma.one - sigma.one %*% solve(sigma.one + gaussian.noise^2 * diag(1, ncol(sigma.one))) %*% sigma.one
# Define mean function and covariance function for training data

In [ ]:
n.samples <- 10  # Define the number of samples for the experiment

In [ ]:
values.one <- matrix(rep(0, length(z) * n.samples), ncol=n.samples)
for (i in 1:n.samples) {
  values.one[, i] <- mvrnorm(1, gp.mean.one, gp.covariance.one)
}
values.one <- cbind(x=z, as.data.frame(values.one))
values.one <- melt(values.one, id="x")
print(values.one)
# Generate sample values for the training data

In [ ]:
g <- data$touchX[301:600] - data$targetX[301:600]  # Define offset for X coordinates 
h <- data$touchY[301:600] - data$targetX[301:600]  # Define offset for Y coordinates
s.star <- c(g, h)  # Combine offset coordinates for Subject15-N9 into single vector

In [ ]:
sigma.two <- CalculateCovariance(z, s.star)
sigma.three <- CalculateCovariance(s.star, z)
sigma.four <- CalculateCovariance(s.star, s.star)
# Define covariance matrix for the new set of input values and the input vector for the training data

In [ ]:
gp.mean.two <- sigma.three %*% solve(sigma.one + gaussian.noise^2 * diag(1, ncol(sigma.one))) %*% z
gp.covariance.two <- sigma.four - sigma.three %*% solve(sigma.one + gaussian.noise^2 * diag(1, ncol(sigma.one))) %*% sigma.two
# Define mean function and covariance function for new set of input values

In [ ]:
values.two <- matrix(rep(0, length(s.star) * n.samples), ncol=n.samples)
for (i in 1:n.samples) {
  values.two[, i] <- mvrnorm(1, gp.mean.two, gp.covariance.two)
}
values.two <- cbind(x=s.star, as.data.frame(values.two))
values.two <- melt(values.two, id="x")
print(values.two)
# Generate sample values for the new set of input values

In [ ]:
figure1 <- plot(values.one$value ~ values.one$x, xlab="Input", ylab="Output", main="Subject15-iPhone4 Touch Accuracy")
figure2 <- plot(values.two$value ~ values.two$x, xlab="Input", ylab="Output", main="Subject15-N9 Touch Accuracy")